<a href="https://colab.research.google.com/github/jimregan/tesseract-gle-uncial/blob/master/Update_gle_uncial_traineddata_for_Tesseract_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grab this for later

In [0]:
!wget https://github.com/jimregan/tesseract-gle-uncial/releases/download/v0.1beta2/gle_uncial.traineddata

Install dependencies

In [0]:
!apt-get install libicu-dev libpango1.0-dev libcairo2-dev libleptonica-dev


Clone, compile and set up Tesseract

In [8]:
!git clone https://github.com/tesseract-ocr/tesseract

Cloning into 'tesseract'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 34087 (delta 0), reused 3 (delta 0), pack-reused 34082
Receiving objects: 100% (34087/34087), 44.79 MiB | 9.99 MiB/s, done.
Resolving deltas: 100% (26460/26460), done.


In [0]:
import os
os.chdir('tesseract')

In [0]:
!sh autogen.sh

In [0]:
!./configure --disable-graphics


In [0]:
!make -j 8
!make install
!ldconfig
!make training
!make training-install

Grab some things to scrape the RIA corpus

In [17]:
import os
os.chdir('/content')
!git clone https://github.com/jimregan/tesseract-gle-uncial/

Cloning into 'tesseract-gle-uncial'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1402 (delta 5), reused 0 (delta 0), pack-reused 1373
Receiving objects: 100% (1402/1402), 200.19 MiB | 13.09 MiB/s, done.
Resolving deltas: 100% (634/634), done.
Checking out files: 100% (630/630), done.


In [0]:
!apt-get install lynx

Scrape the RIA corpus

In [0]:
! for i in A B C D E F G H I J K L M N O P Q R S T U V W X Y Z;do lynx -dump "http://corpas.ria.ie/index.php?fsg_function=1&fsg_page=$i" |grep http://corpas.ria.ie|awk '{print $NF}' >> list;done

In [0]:
!grep 'function=3' list |sort|uniq|grep corpas.ria|sed -e 's/function=3/function=5/' > input

In [0]:
!wget -x -c -i input

In [0]:
!mkdir text
!for i in corpas.ria.ie/*;do id=$(echo $i|awk -F'=' '{print $NF}');cat $i | perl /content/tesseract-gle-uncial/scripts/extract-ria.pl > text/$id.txt;done

Get the raw corpus in a single text file

In [0]:
!cat text/*.txt|grep -v '^$' > ria-raw.txt


Compress the raw text; this can be downloaded through the file browser on the left, so the scraping steps can be skipped in future

In [0]:
!gzip ria-raw.txt

...and can be re-added using the upload feature in the file browser

In [0]:
!gzip -d ria-raw.txt.gz

This next part is so I can update the langdata files

In [33]:
import os
os.chdir('/content')
!git clone https://github.com/tesseract-ocr/langdata

Cloning into 'langdata'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2164 (delta 9), reused 12 (delta 2), pack-reused 2129
Receiving objects: 100% (2164/2164), 400.19 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (366/366), done.
Checking out files: 100% (1022/1022), done.


In [0]:
!cat ria-raw.txt | perl /content/tesseract-gle-uncial/scripts/toponc.pl > ria-ponc.txt

In [0]:
!mkdir genwlout

In [0]:
!perl /content/tesseract-gle-uncial/scripts/genlangdata.pl -i ria-ponc.txt -d genwlout -p gle_uncial

In [0]:
import os
os.chdir('/content/genwlout')
#!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cat $i.unsorted | awk -F'\t' '{print $1}' | sort | uniq > $i.sorted;done
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cat $i.sorted /content/langdata/gle_uncial/$i | sort | uniq > $i;done

In [0]:
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cp $i /content/langdata/gle_uncial/;done

In [0]:
Grab the fonts

In [0]:
import os
os.chdir('/content')
!mkdir fonts
os.chdir('fonts')
!wget -i /content/tesseract-gle-uncial/fonts.txt

In [0]:
!for i in *.zip; do unzip $i;done

Generate

In [51]:
os.chdir('/content')
!mkdir unpack
!combine_tessdata -u /content/gle_uncial.traineddata unpack/gle_uncial.

Extracting tessdata components from /content/gle_uncial.traineddata
Wrote unpack/gle_uncial.lstm
Wrote unpack/gle_uncial.lstm-punc-dawg
Wrote unpack/gle_uncial.lstm-word-dawg
Wrote unpack/gle_uncial.lstm-number-dawg
Wrote unpack/gle_uncial.lstm-unicharset
Wrote unpack/gle_uncial.lstm-recoder
Wrote unpack/gle_uncial.version
Version string:4.00.00alpha
17:lstm:size=4291340, offset=192
18:lstm-punc-dawg:size=890, offset=4291532
19:lstm-word-dawg:size=5760002, offset=4292422
20:lstm-number-dawg:size=226, offset=10052424
21:lstm-unicharset:size=9938, offset=10052650
22:lstm-recoder:size=1147, offset=10062588
23:version:size=12, offset=10063735


In [0]:
os.chdir('unpack')
!for i in gle_uncial.word.bigrams gle_uncial.wordlist gle_uncial.numbers gle_uncial.punc; do cp /content/genwlout/$i .;done

In [57]:
!ls -al


total 102304
drwxr-xr-x 2 root root     4096 Apr 19 15:23 .
drwxr-xr-x 1 root root     4096 Apr 19 15:20 ..
-rw-r--r-- 1 root root  4291340 Apr 19 15:20 gle_uncial.lstm
-rw-r--r-- 1 root root    61530 Apr 19 15:30 gle_uncial.lstm-number-dawg
-rw-r--r-- 1 root root      890 Apr 19 15:20 gle_uncial.lstm-punc-dawg
-rw-r--r-- 1 root root     1147 Apr 19 15:20 gle_uncial.lstm-recoder
-rw-r--r-- 1 root root     9938 Apr 19 15:20 gle_uncial.lstm-unicharset
-rw-r--r-- 1 root root  5760002 Apr 19 15:20 gle_uncial.lstm-word-dawg
-rw-r--r-- 1 root root    41842 Apr 19 15:23 gle_uncial.numbers
-rw-r--r-- 1 root root    26150 Apr 19 15:23 gle_uncial.punc
-rw-r--r-- 1 root root       12 Apr 19 15:20 gle_uncial.version
-rw-r--r-- 1 root root 80683543 Apr 19 15:23 gle_uncial.word.bigrams
-rw-r--r-- 1 root root 13840791 Apr 19 15:23 gle_uncial.wordlist


In [58]:
!wordlist2dawg gle_uncial.numbers  gle_uncial.lstm-number-dawg gle_uncial.lstm-unicharset
!wordlist2dawg gle_uncial.punc  gle_uncial.lstm-punc-dawg gle_uncial.lstm-unicharset
!wordlist2dawg gle_uncial.wordlist  gle_uncial.lstm-word-dawg gle_uncial.lstm-unicharset

Loading unicharset from 'gle_uncial.lstm-unicharset'
Reading word list from 'gle_uncial.numbers'
Reducing Trie to SquishedDawg
Writing squished DAWG to 'gle_uncial.lstm-number-dawg'
Loading unicharset from 'gle_uncial.lstm-unicharset'
Reading word list from 'gle_uncial.punc'
Reducing Trie to SquishedDawg
Writing squished DAWG to 'gle_uncial.lstm-punc-dawg'
Loading unicharset from 'gle_uncial.lstm-unicharset'
Reading word list from 'gle_uncial.wordlist'
Reducing Trie to SquishedDawg
Writing squished DAWG to 'gle_uncial.lstm-word-dawg'


In [59]:
!ls -al

total 107788
drwxr-xr-x 2 root root     4096 Apr 19 15:23 .
drwxr-xr-x 1 root root     4096 Apr 19 15:20 ..
-rw-r--r-- 1 root root  4291340 Apr 19 15:20 gle_uncial.lstm
-rw-r--r-- 1 root root    61530 Apr 19 15:31 gle_uncial.lstm-number-dawg
-rw-r--r-- 1 root root    24834 Apr 19 15:31 gle_uncial.lstm-punc-dawg
-rw-r--r-- 1 root root     1147 Apr 19 15:20 gle_uncial.lstm-recoder
-rw-r--r-- 1 root root     9938 Apr 19 15:20 gle_uncial.lstm-unicharset
-rw-r--r-- 1 root root 11350106 Apr 19 15:31 gle_uncial.lstm-word-dawg
-rw-r--r-- 1 root root    41842 Apr 19 15:23 gle_uncial.numbers
-rw-r--r-- 1 root root    26150 Apr 19 15:23 gle_uncial.punc
-rw-r--r-- 1 root root       12 Apr 19 15:20 gle_uncial.version
-rw-r--r-- 1 root root 80683543 Apr 19 15:23 gle_uncial.word.bigrams
-rw-r--r-- 1 root root 13840791 Apr 19 15:23 gle_uncial.wordlist
